In [1]:
import torch 

In [2]:
torch.version

<module 'torch.version' from '/home/lab/anaconda3/envs/nsvqa/lib/python3.6/site-packages/torch/version.py'>

In [3]:
import sys
sys.path.append('/home/lab/A305/yifan/vqa/vqa-exp/VQA_E_VCR/')
from dataloaders.vcr import VCR, VCRLoader

/home/lab/A305/yifan/vqa/vqa-exp/VQA_E_VCR/data/vcr1images


In [4]:
"""
Training script. Should be pretty adaptable to whatever.
"""
import argparse
import os
import shutil

import multiprocessing
import numpy as np
import pandas as pd
import torch
from allennlp.common.params import Params
from allennlp.training.learning_rate_schedulers import LearningRateScheduler
from allennlp.training.optimizers import Optimizer
from torch.nn import DataParallel
from torch.nn.modules import BatchNorm2d
from tqdm import tqdm

from dataloaders.vcr import VCR, VCRLoader
from utils.pytorch_misc import time_batch, save_checkpoint, clip_grad_norm, \
    restore_checkpoint, print_para, restore_best_checkpoint

import logging
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s - %(message)s', level=logging.DEBUG)

# This is needed to make the imports work
from allennlp.models import Model
import models
# import sys
# sys.path.append("/home/lab/A305/yifan/vqa/vqa-exp/r2c/")

In [5]:
#################################
#################################
######## Data loading stuff
#################################
#################################

parser = argparse.ArgumentParser(description='train')
parser.add_argument(
    '-params',
    dest='params',
    help='Params location',
    type=str,
)
parser.add_argument(
    '-rationale',
    action="store_true",
    help='use rationale',
)
parser.add_argument(
    '-folder',
    dest='folder',
    help='folder location',
    type=str,
)
parser.add_argument(
    '-no_tqdm',
    dest='no_tqdm',
    action='store_true',
)

_StoreTrueAction(option_strings=['-no_tqdm'], dest='no_tqdm', nargs=0, const=True, default=False, type=None, choices=None, help=None, metavar=None)

In [6]:
args = parser.parse_args()
args.params = "multiatt_scene/default.json"
args.folder = "saves/multiatt_scene/flagship_answer"
args.no_tqdm = True

params = Params.from_file(args.params)
print(args.no_tqdm)
# print(params.values)
# param=params['model']
# print(param)
# print(param['type'])
embs_to_load=params['dataset_reader'].get('embs', 'bert_da')
print(embs_to_load)
print(type(embs_to_load))
# type(params['dataset_reader'])
params['dataset_reader'].keys()

True
bert_da
<class 'str'>


KeysView(<allennlp.common.params.Params object at 0x7fe0ce46be48>)

In [7]:
train, val, test = VCR.splits(mode='rationale' if args.rationale else 'answer',
                              embs_to_load=params['dataset_reader'].get('embs', 'bert_da'),
                              only_use_relevant_dets=params['dataset_reader'].get('only_use_relevant_dets', True))
NUM_GPUS = torch.cuda.device_count()
NUM_CPUS = multiprocessing.cpu_count()
if NUM_GPUS == 0:
    raise ValueError("you need gpus!")

def _to_gpu(td):
    if NUM_GPUS > 1:
        return td
    for k in td:
        if k != 'metadata':
            td[k] = {k2: v.cuda(non_blocking=True) for k2, v in td[k].items()} if isinstance(td[k], dict) else td[k].cuda(
                non_blocking=True)
    return td
num_workers = (4 * NUM_GPUS if NUM_CPUS >= 32 else 2*NUM_GPUS)-1
#print(f"Using {num_workers} workers out of {NUM_CPUS} possible", flush=True)
# print(f"Using {num_workers} workers out of {NUM_CPUS} possible")

Only relevant dets
Loading embeddings from /home/lab/A305/yifan/vqa/vqa-exp/VQA_E_VCR/data/bert_da_answer_train.h5
Only relevant dets
Loading embeddings from /home/lab/A305/yifan/vqa/vqa-exp/VQA_E_VCR/data/bert_da_answer_val.h5
Only relevant dets
Loading embeddings from /home/lab/A305/yifan/vqa/vqa-exp/VQA_E_VCR/data/bert_da_answer_test.h5


In [8]:
loader_params = {'batch_size': 96 // NUM_GPUS, 'num_gpus':NUM_GPUS, 'num_workers':num_workers}
train_loader = VCRLoader.from_dataset(train, **loader_params)
val_loader = VCRLoader.from_dataset(val, **loader_params)
test_loader = VCRLoader.from_dataset(test, **loader_params)

ARGS_RESET_EVERY = 100
#print("Loading {} for {}".format(params['model'].get('type', 'WTF?'), 'rationales' if args.rationale else 'answer'), flush=True)
print("Loading {} for {}".format(params['model'].get('type', 'WTF?'), 'rationales' if args.rationale else 'answer'))


Loading MultiAttentionQA_Scene for answer


In [9]:
model = Model.from_params(vocab=train.vocab, params=params['model'])

03/27/2019 01:24:06 - INFO - allennlp.common.from_params -   instantiating class <class 'allennlp.models.model.Model'> from params {'hidden_dim_maxpool': 1024, 'initializer': [['.*final_mlp.*weight', {'type': 'xavier_uniform'}], ['.*final_mlp.*bias', {'type': 'zero'}], ['.*weight_ih.*', {'type': 'xavier_uniform'}], ['.*weight_hh.*', {'type': 'orthogonal'}], ['.*bias_ih.*', {'type': 'zero'}], ['.*bias_hh.*', {'type': 'lstm_hidden_bias'}]], 'input_dropout': 0.3, 'pool_answer': True, 'pool_question': True, 'reasoning_encoder': {'bidirectional': True, 'hidden_size': 256, 'input_size': 1536, 'num_layers': 2, 'type': 'lstm'}, 'span_encoder': {'bidirectional': True, 'hidden_size': 256, 'input_size': 768, 'num_layers': 1, 'type': 'lstm'}, 'type': 'MultiAttentionQA_Scene'} and extras {'vocab': <allennlp.data.vocabulary.Vocabulary object at 0x7fe038946048>}
03/27/2019 01:24:06 - INFO - allennlp.common.params -   model.type = MultiAttentionQA_Scene
03/27/2019 01:24:06 - INFO - allennlp.common.fro

03/27/2019 01:24:06 - INFO - allennlp.common.params -   CURRENTLY DEFINED PARAMETERS: 
03/27/2019 01:24:06 - INFO - allennlp.common.from_params -   instantiating class <class 'allennlp.nn.initializers.Initializer'> from params {'type': 'lstm_hidden_bias'} and extras {}
03/27/2019 01:24:06 - INFO - allennlp.common.params -   model.initializer.list.list.type = lstm_hidden_bias
03/27/2019 01:24:06 - INFO - allennlp.common.params -   Converting Params object to dict; logging of default values will not occur when dictionary parameters are used subsequently.
03/27/2019 01:24:06 - INFO - allennlp.common.params -   CURRENTLY DEFINED PARAMETERS: 


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~init_0~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


/home/lab/anaconda3/envs/nsvqa/lib/python3.6/site-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
03/27/2019 01:24:07 - INFO - allennlp.nn.initializers -   Initializing parameters
03/27/2019 01:24:07 - INFO - allennlp.nn.initializers -   Initializing span_encoder._module._module.weight_ih_l0 using .*weight_ih.* intitializer
03/27/2019 01:24:07 - INFO - allennlp.nn.initializers -   Initializing span_encoder._module._module.weight_hh_l0 using .*weight_hh.* intitializer
03/27/2019 01:24:07 - INFO - allennlp.nn.initializers -   Initializing span_encoder._module._module.bias_ih_l0 using .*bias_ih.* intitializer
03/27/2019 01:24:07 - INFO - allennlp.nn.initializers -   Initializing span_encoder._module._module.bias_hh_l0 using .*bias_hh.* intitializer
03/27/2019 01:24:07 - INFO - allennlp.nn.initializers -   Initializing span_encoder._module._module.weight_ih_l0_reverse 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~init_0.1~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~init_0.2~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
init_1
init_2
init_3


03/27/2019 01:24:07 - INFO - allennlp.nn.initializers -   Initializing reasoning_encoder._module._module.weight_hh_l1 using .*weight_hh.* intitializer
03/27/2019 01:24:07 - INFO - allennlp.nn.initializers -   Initializing reasoning_encoder._module._module.bias_ih_l1 using .*bias_ih.* intitializer
03/27/2019 01:24:07 - INFO - allennlp.nn.initializers -   Initializing reasoning_encoder._module._module.bias_hh_l1 using .*bias_hh.* intitializer
03/27/2019 01:24:07 - INFO - allennlp.nn.initializers -   Initializing reasoning_encoder._module._module.weight_ih_l1_reverse using .*weight_ih.* intitializer
03/27/2019 01:24:07 - INFO - allennlp.nn.initializers -   Initializing reasoning_encoder._module._module.weight_hh_l1_reverse using .*weight_hh.* intitializer
03/27/2019 01:24:07 - INFO - allennlp.nn.initializers -   Initializing reasoning_encoder._module._module.bias_ih_l1_reverse using .*bias_ih.* intitializer
03/27/2019 01:24:07 - INFO - allennlp.nn.initializers -   Initializing reasoning_e

03/27/2019 01:24:07 - INFO - allennlp.nn.initializers -      detector.backbone.5.0.bn3.bias
03/27/2019 01:24:07 - INFO - allennlp.nn.initializers -      detector.backbone.5.0.bn3.weight
03/27/2019 01:24:07 - INFO - allennlp.nn.initializers -      detector.backbone.5.0.conv1.weight
03/27/2019 01:24:07 - INFO - allennlp.nn.initializers -      detector.backbone.5.0.conv2.weight
03/27/2019 01:24:07 - INFO - allennlp.nn.initializers -      detector.backbone.5.0.conv3.weight
03/27/2019 01:24:07 - INFO - allennlp.nn.initializers -      detector.backbone.5.0.downsample.0.weight
03/27/2019 01:24:07 - INFO - allennlp.nn.initializers -      detector.backbone.5.0.downsample.1.bias
03/27/2019 01:24:07 - INFO - allennlp.nn.initializers -      detector.backbone.5.0.downsample.1.weight
03/27/2019 01:24:07 - INFO - allennlp.nn.initializers -      detector.backbone.5.1.bn1.bias
03/27/2019 01:24:07 - INFO - allennlp.nn.initializers -      detector.backbone.5.1.bn1.weight
03/27/2019 01:24:07 - INFO - alle

03/27/2019 01:24:07 - INFO - allennlp.nn.initializers -      detector.backbone.6.5.bn3.bias
03/27/2019 01:24:07 - INFO - allennlp.nn.initializers -      detector.backbone.6.5.bn3.weight
03/27/2019 01:24:07 - INFO - allennlp.nn.initializers -      detector.backbone.6.5.conv1.weight
03/27/2019 01:24:07 - INFO - allennlp.nn.initializers -      detector.backbone.6.5.conv2.weight
03/27/2019 01:24:07 - INFO - allennlp.nn.initializers -      detector.backbone.6.5.conv3.weight
03/27/2019 01:24:07 - INFO - allennlp.nn.initializers -      detector.mask_upsample.bias
03/27/2019 01:24:07 - INFO - allennlp.nn.initializers -      detector.mask_upsample.weight
03/27/2019 01:24:07 - INFO - allennlp.nn.initializers -      detector.obj_downsample.1.bias
03/27/2019 01:24:07 - INFO - allennlp.nn.initializers -      detector.obj_downsample.1.weight
03/27/2019 01:24:07 - INFO - allennlp.nn.initializers -      detector.object_embed.weight
03/27/2019 01:24:07 - INFO - allennlp.nn.initializers -      detector.

In [10]:
for submodule in model.detector.backbone.modules():
    if isinstance(submodule, BatchNorm2d):
        submodule.track_running_stats = False
    for p in submodule.parameters():
        p.requires_grad = False
model = DataParallel(model).cuda() if NUM_GPUS > 1 else model.cuda()

optimizer = Optimizer.from_params([x for x in model.named_parameters() if x[1].requires_grad],
                                  params['trainer']['optimizer'])


03/27/2019 01:24:11 - INFO - allennlp.common.params -   trainer.optimizer.type = adam
03/27/2019 01:24:11 - INFO - allennlp.common.params -   trainer.optimizer.parameter_groups = None
03/27/2019 01:24:11 - INFO - allennlp.training.optimizers -   Number of trainable parameters: 25707540
03/27/2019 01:24:11 - INFO - allennlp.common.params -   Converting Params object to dict; logging of default values will not occur when dictionary parameters are used subsequently.
03/27/2019 01:24:11 - INFO - allennlp.common.params -   CURRENTLY DEFINED PARAMETERS: 
03/27/2019 01:24:11 - INFO - allennlp.common.params -   trainer.optimizer.lr = 0.0002
03/27/2019 01:24:11 - INFO - allennlp.common.params -   trainer.optimizer.weight_decay = 0.0001


In [11]:
lr_scheduler_params = params['trainer'].pop("learning_rate_scheduler", None)
scheduler = LearningRateScheduler.from_params(optimizer, lr_scheduler_params) if lr_scheduler_params else None
if os.path.exists(args.folder):
    #print("Found folder! restoring", flush=True)
    print("Found folder! restoring")
    start_epoch, val_metric_per_epoch = restore_checkpoint(model, optimizer, serialization_dir=args.folder,
                                                           learning_rate_scheduler=scheduler)
else:
    print("Making directories")
    os.makedirs(args.folder, exist_ok=True)
    start_epoch, val_metric_per_epoch = 0, []
    shutil.copy2(args.params, args.folder)


03/27/2019 01:24:11 - INFO - allennlp.common.params -   trainer.learning_rate_scheduler.type = reduce_on_plateau
03/27/2019 01:24:11 - INFO - allennlp.common.params -   Converting Params object to dict; logging of default values will not occur when dictionary parameters are used subsequently.
03/27/2019 01:24:11 - INFO - allennlp.common.params -   CURRENTLY DEFINED PARAMETERS: 
03/27/2019 01:24:11 - INFO - allennlp.common.params -   trainer.learning_rate_scheduler.cooldown = 2
03/27/2019 01:24:11 - INFO - allennlp.common.params -   trainer.learning_rate_scheduler.factor = 0.5
03/27/2019 01:24:11 - INFO - allennlp.common.params -   trainer.learning_rate_scheduler.mode = max
03/27/2019 01:24:11 - INFO - allennlp.common.params -   trainer.learning_rate_scheduler.patience = 1
03/27/2019 01:24:11 - INFO - allennlp.common.params -   trainer.learning_rate_scheduler.verbose = True


Found folder! restoring


In [14]:
param_shapes = print_para(model)
num_batches = 0


 34.3M total parameters. 25.7M training 
 ----- 
                                                               shape     size  requires_grad
name                                                                                       
detector.after_roi_align.0.0.conv2.weight             [512,512,3,3]  2359296           True
detector.after_roi_align.0.1.conv2.weight             [512,512,3,3]  2359296           True
detector.after_roi_align.0.2.conv2.weight             [512,512,3,3]  2359296           True
detector.after_roi_align.0.0.downsample.0.weight    [2048,1024,1,1]  2097152           True
reasoning_encoder._module._module.weight_ih_l0          [1024,1536]  1572864           True
reasoning_encoder._module._module.weight_ih_l0_...      [1024,1536]  1572864           True
final_mlp.1.weight                                      [1024,1536]  1572864           True
detector.obj_downsample.1.weight                         [512,2176]  1114112           True
detector.after_roi_align.0.0.

In [15]:
for epoch_num in range(start_epoch, params['trainer']['num_epochs'] + start_epoch):
    train_results = []
    norms = []
    model.train()
    print('1111')
    for b, (time_per_batch, batch) in enumerate(time_batch(train_loader if args.no_tqdm else tqdm(train_loader), reset_every=ARGS_RESET_EVERY)):
        print('2222')
        batch = _to_gpu(batch)
        print('3333')
        optimizer.zero_grad()
        print('4444')
        output_dict = model(**batch)
        print('5555')
        loss = output_dict['loss'].mean() + output_dict['cnn_regularization_loss'].mean()
        print('6666')
        loss.backward()
        print('7777')
        num_batches += 1
        if scheduler:
            scheduler.step_batch(num_batches)
        print('8888')
        norms.append(
            clip_grad_norm(model.named_parameters(), max_norm=params['trainer']['grad_norm'], clip=True, verbose=False)
        )
        print('9999')
        optimizer.step()
        print('0000')
        exit()
    exit()
sys.exit()

1111
2222
3333
4444
<class 'torch.Tensor'>
<class 'dict'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
#######################images##################
torch.Size([96, 3, 384, 768])
#######################question##################
1
bert torch.Size([96, 4, 18, 768])
#######################question_tags##################
<class 'torch.Tensor'>
torch.Size([96, 4, 18])
#######################question_mask##################
torch.Size([96, 4, 18])


ValueError: 11111Stop by the user

In [4]:
import torch
from torch.autograd import Variable as V
import torchvision.models as models
from torchvision import transforms as trn
from torch.nn import functional as F
import os
from PIL import Image

from functools import partial
import pickle
# from torch.utils.serialization import load_lua


# th architecture to use
arch = 'resnet50'
# arch = 'resnet152'

# load the pre-trained weights

model_file = '%s_places365.pth.tar' % arch
# model_file = '%s_places365.t7' % arch


print(os.access(model_file, os.W_OK))


# pickle.load = partial(pickle.load, encoding="latin1")
# pickle.Unpickler = partial(pickle.Unpickler, encoding="latin1")
# model = torch.load(model_file, map_location=lambda storage, loc: storage, pickle_module=pickle)


./models/resnet50_places365.pth.tar
True


In [5]:
if not os.access(model_file, os.W_OK):
    print('Downloading resnet50_places365.pth.tar')
    weight_url = 'http://places2.csail.mit.edu/models_places365/' + model_file
    os.system('wget ' + weight_url)
    print('Finished')

In [6]:
model = models.__dict__[arch](num_classes=365)

In [8]:
checkpoint = torch.load(model_file, map_location=lambda storage, loc: storage)
state_dict = {str.replace(k,'module.',''): v for k,v in checkpoint['state_dict'].items()}
model.load_state_dict(state_dict)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [9]:
# load the image transformer
from torchvision import transforms as trn
centre_crop = trn.Compose([
        trn.Resize((256,256)),
        trn.CenterCrop(224),
        trn.ToTensor(),
        trn.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [10]:
# load the class label
file_name = 'categories_places365.txt'
if not os.access(file_name, os.W_OK):
    synset_url = 'https://raw.githubusercontent.com/csailvision/places365/master/categories_places365.txt'
    os.system('wget ' + synset_url)
classes = list()
with open(file_name) as class_file:
    for line in class_file:
        classes.append(line.strip().split(' ')[0][3:])
classes = tuple(classes)

In [11]:
# load the test image
img_name = '12.jpg'
if not os.access(img_name, os.W_OK):
    img_url = 'http://places.csail.mit.edu/demo/' + img_name
    os.system('wget ' + img_url)

In [18]:
img = Image.open(img_name)
print(type(img))
temp = centre_crop(img).unsqueeze(0)
print(type(temp))
print(temp.size())
input_img = V(temp)
print(type(input_img))
print(input_img.size())

<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'torch.Tensor'>
torch.Size([1, 3, 224, 224])
<class 'torch.Tensor'>
torch.Size([1, 3, 224, 224])


In [13]:
# forward pass
logit = model.forward(input_img)
h_x = F.softmax(logit, 1).data.squeeze()
probs, idx = h_x.sort(0, True)

print('{} prediction on {}'.format(arch,img_name))
# output the prediction
for i in range(0, 5):
    print('{:.3f} -> {}'.format(probs[i], classes[idx[i]]))

resnet50 prediction on 12.jpg
0.685 -> patio
0.240 -> restaurant_patio
0.019 -> beer_garden
0.010 -> courtyard
0.010 -> porch


In [20]:

import torch
import torch.nn as nn
import torch.nn.parallel
# from torchvision.models import resnet

# from utils.pytorch_misc import Flattener
# from torchvision.layers import ROIAlign
# import torch.utils.model_zoo as model_zoo
USE_IMAGENET_PRETRAINED = True
# from utils.pytorch_misc import pad_sequence


from torch.autograd import Variable as V
import torchvision.models as models
from torchvision import transforms as trn
from torch.nn import functional as F
import os
from PIL import Image
from functools import partial
import pickle



def _load_resnet(pretrained=True):
    # huge thx to https://github.com/ruotianluo/pytorch-faster-rcnn/blob/master/lib/nets/resnet_v1.py
    backbone = resnet.resnet50(pretrained=False)
    if pretrained:
        backbone.load_state_dict(model_zoo.load_url(
            'https://s3.us-west-2.amazonaws.com/ai2-rowanz/resnet50-e13db6895d81.th'))
    for i in range(2, 4):
        getattr(backbone, 'layer%d' % i)[0].conv1.stride = (2, 2)
        getattr(backbone, 'layer%d' % i)[0].conv2.stride = (1, 1)
    return backbone


def _load_resnet_imagenet(pretrained=True):
    # huge thx to https://github.com/ruotianluo/pytorch-faster-rcnn/blob/master/lib/nets/resnet_v1.py

    
    backbone = resnet.resnet50(pretrained=pretrained)
    
    
    for i in range(2, 4):
        getattr(backbone, 'layer%d' % i)[0].conv1.stride = (2, 2)
        getattr(backbone, 'layer%d' % i)[0].conv2.stride = (1, 1)
    # use stride 1 for the last conv4 layer (same as tf-faster-rcnn)
    backbone.layer4[0].conv2.stride = (1, 1)
    backbone.layer4[0].downsample[0].stride = (1, 1)

    # # Make batchnorm more sensible
    # for submodule in backbone.modules():
    #     if isinstance(submodule, torch.nn.BatchNorm2d):
    #         submodule.momentum = 0.01

    return backbone

In [22]:
from torchvision.models import resnet

In [24]:
backbone = resnet.resnet50(pretrained=True)

In [30]:
#backbone

In [28]:
for i in range(2, 4):
    print()
    getattr(backbone, 'layer%d' % i)[0].conv1.stride = (2, 2)
    getattr(backbone, 'layer%d' % i)[0].conv2.stride = (1, 1)


In [43]:
images = torch.randn([96, 3, 384, 768])
org_size = images.size()
m = nn.Upsample(size= [org_size[0],org_size[1],256,256],mode = 'bilinear')
new_images = m(images)

/home/lab/anaconda3/envs/nsvqa/lib/python3.6/site-packages/torch/nn/functional.py:2423: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


RuntimeError: Expected a list of 2 ints but got 4 for argument #2 'output_size'

In [31]:
centre_crop = trn.Compose([
            trn.Resize((256,256)),
            trn.CenterCrop(224),
            trn.ToTensor(),
            trn.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [35]:
size = images.size()
images = images.resize([size[0],size[1],256,256])
images.size()

/home/lab/anaconda3/envs/nsvqa/lib/python3.6/site-packages/torch/tensor.py:287: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


RuntimeError: requested resize to [96, 3, 256, 256] ([96, 3, 256, 256] elements in total), but the given tensor has a size of 96x3x384x768 (84934656 elements). autograd's resize can only change the shape of a given tensor, while preserving the number of elements. 

In [ ]:
temp = centre_crop(images)
input_img = V(temp.unsqueeze(0))

In [36]:
input = torch.arange(1, 5).view(1, 1, 2, 2).float()

In [37]:
input

tensor([[[[1., 2.],
          [3., 4.]]]])

In [38]:
 m = nn.Upsample(sizescale_factor=2, mode='nearest')

In [39]:
m(input)

/home/lab/anaconda3/envs/nsvqa/lib/python3.6/site-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))


tensor([[[[1., 1., 2., 2.],
          [1., 1., 2., 2.],
          [3., 3., 4., 4.],
          [3., 3., 4., 4.]]]])

In [40]:
input_3x3 = torch.zeros(3, 3).view(1, 1, 3, 3)

In [41]:
input

tensor([[[[1., 2.],
          [3., 4.]]]])